In [62]:
# import os, getpass
import chromadb
from ibm_watsonx_ai import Credentials, APIClient
from langchain_chroma import Chroma
from langchain_core.documents import Document
from uuid import uuid4
from langchain_text_splitters import MarkdownTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import ChatWatsonx
# from langchain.chains import RetrievalQAjin
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import AutoTokenizer
from docling.document_converter import DocumentConverter
from utils import *
PAGE_BREAK = "<!-- page break -->"

In [2]:
api_key="VPryyL4t508YQ_pJQNCU0m23Yctctahrqo2pjT5BMQkV"
url="https://us-south.ml.cloud.ibm.com"
project_id = "6552c3bb-a35a-4bf3-8430-c42828d59a06"

## Load documents

In [3]:
## TODO preserve metadata e.g. page number, chunking should probably be done per page

file_path = "data/climate_edu/389801eng.pdf"
converter = DocumentConverter()
result = converter.convert(file_path)
doc = result.document


/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(


In [52]:
doc_name = doc.name

In [25]:
doc_md = doc.export_to_markdown(page_break_placeholder=PAGE_BREAK, image_placeholder="",)
with open("data/climate_edu/389801eng.md", "w") as f:
    f.write(doc_md)

pages = doc_md.split(PAGE_BREAK)
pages = [markdown_cleanup(page) for page in pages]


## Chunk

In [87]:
## TODO change model to granite embed
tokenizer = AutoTokenizer.from_pretrained(
    "ibm-granite/granite-embedding-278m-multilingual"
)
max_length = tokenizer.model_max_length
markdown_splitter = MarkdownTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,
    chunk_size=max_length,
    chunk_overlap=int(max_length/10)
)

In [ ]:
chunked_docs = []
for i, page in enumerate(pages):
    chunks = markdown_splitter.split_text(page)
    for j, chunk in enumerate(chunks):
        document = Document(
            page_content=chunk,
            embeddings=tokenizer.encode(chunk, add_special_tokens=False),
            metadata={
                "source": doc_name,
                "page": i + 1,
                "chunk": j + 1
            }
        )
        chunked_docs.append(document)

## Ingest

In [84]:
embedder = WatsonxEmbeddings(
    url=url,
    apikey=api_key,
    project_id=project_id,
    model_id="ibm/granite-embedding-278m-multilingual"
)

In [79]:
vector_store = Chroma(
    collection_name="climate_edu",
    persist_directory="data/chroma_db",
    embedding_function=embedder
)

In [ ]:
vector_store.add_documents(chunked_docs)

['00607966-6456-4f0e-a331-93a41256c163',
 '93a69a3f-2bfd-41cf-884e-05e51172b5b5',
 '1da11a6e-bf76-48d2-8c0a-7a0c2f200638',
 '3895b380-9e4d-4491-bafa-16631560ff45',
 '9826493a-b694-40f8-be1e-d137acd12802',
 '6f9eb590-69bb-4e9d-8a74-7747f2fec488',
 'a36d494f-fcdb-4285-9c5c-225873caf261',
 'fe482308-ef9c-4309-af17-56628babfede',
 '354e98e1-ff42-447c-9121-3962ee176036',
 '08c5445d-afaf-4744-8b70-21dfcfcbb92a',
 'c7f91fb6-72b3-431b-b588-d2fcd31f29bb',
 '394c854c-5242-4723-8e3a-8944a6b9b2ec',
 '2a0ec83d-da9a-47ab-b9c7-475eccf36709',
 'be610ffe-f2ff-4960-b99c-b4847d5ad277',
 'a15a6ae6-6583-4a14-8061-ee494dae4fd1',
 '1326d1a0-9021-4223-b54f-fb32e28be181',
 'a0d7c80c-2b1b-4347-ad0e-fc6928d15a1b',
 '5505c6c3-967c-421e-bcc0-fd9170ddd51b',
 'd48474b6-c29b-4b99-965f-876a396f19a4',
 'ff35aec6-4eeb-4115-8e20-7a68fae73630',
 'c0cc3710-18ce-4439-baf8-ebb3a6496e7d',
 '8cd31040-9415-4f58-8712-3fa69925b0ab',
 '8df25092-121d-426a-be0a-2dd09010e476',
 'f18ff793-38e8-4c58-bebc-2f1593132ff9',
 '63874ef1-072f-

## Retrieval

In [81]:
results = vector_store.similarity_search(
    "how many people are affected by climate change?",
    k=2
)

## Augmentation

In [ ]:
## TODO prompt template for retrieval QA

[Document(id='5635ed43-8967-4de4-87c1-65e933ee5cd4', metadata={'page': 18, 'chunk': 2, 'source': '389801eng'}, page_content="Individuals can experience a range of emotions related to climate change, from hope to anxiety and from grief to denial (Pihkala, 2022). Increased knowledge can also lead to further anxiety about climate change, when not accompanied by addressing these feelings and offering opportunities for engaging with climate action (Hargis and McKenzie, 2020). A study in the US state of North Carolina showed that climate change hope and concern were\n## FIGURE 4:\nMost students are taught about climate change in school in richer countries but their scientific knowledge and concern for the environment vary\nCoverage of climate change in the secondary school curriculum and 15-year-old students' concern for the environment, 2018\nSource : GEM Report team calculations based on the 2018 PISA.\npositively related to behaviour, while feeling despair was negatively related to behavi

In [ ]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"
parameters = TextChatParameters(
    max_tokens = 1024,
    temperature = 0
)
watsonx_llm = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters,
)

In [ ]:
messages = [
    ("system", "You are a helpful assistant that translates English to French."),
    (
        "human",
        "I love you for listening to Rock.",
    ),
]


In [ ]:

out = watsonx_llm.invoke(messages)


In [ ]:
out.content